step 1 : install
        -> pip install visdom

step 2 :  run server damon

In [3]:
from visdom import Visdom

In [8]:
# train example

import torch
import numpy as np
from torch import nn
import torch.nn.functional as F
from torchvision import datasets, transforms

# params
batch_size = 256
learning_rate = 1e-3
epochs = 10

# transform
trans = transforms.ToTensor()

# initialize the train and test datasets
train_loader = torch.utils.data.DataLoader(datasets.FashionMNIST(root='data', 
                                                                 train=True, 
                                                                 transform=trans, 
                                                                 download=True), 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(datasets.FashionMNIST(root='data', 
                                                                train=False, 
                                                                transform=trans, 
                                                                download=True), 
                                          batch_size=batch_size)

# network set
w1, b1 = torch.randn(200,784,requires_grad=True),\
        torch.zeros(200,requires_grad=True)
w2, b2 = torch.randn(200,200,requires_grad=True),\
        torch.zeros(200, requires_grad=True)
w3, b3 = torch.randn(10,200,requires_grad=True),\
        torch.zeros(10,requires_grad=True)

# init
torch.nn.init.kaiming_normal_(w1)
torch.nn.init.kaiming_normal_(w2)
torch.nn.init.kaiming_normal_(w3)

# forward
def forward(x):
    x = x@w1.T + b1
    x = F.relu(x)
    
    x = x@w2.T + b2
    x = F.relu(x)
    
    x = x@w3.T + b3
    x = F.softmax(x)
    return x

# optimizer
optimizer = torch.optim.SGD([w1,w2,w3,b1,b2,b3], lr=learning_rate)

# loss function
loss_fn = nn.CrossEntropyLoss()

#-->visdom可视化<--
viz = Visdom()
viz.line([0.], [0.], win='train_loss', opts=dict(title='train_loss')) # init
viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test_loss & acc.', 
                                                   legend=['loss', 'acc.']))

# 记录步长
global_step = 0

# training
for epoch in range(epochs):
    # forward
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.reshape(-1, 28*28)
        y_hat = forward(data)
        
        loss = loss_fn(y_hat, target)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        global_step += 1
        # 可视化矩阵
        viz.line([loss.item()], [global_step], win='train_loss', update='append')
        
        if batch_idx%100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))
    
    # test
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data = data.reshape(-1,28*28)
        
        logits = forward(data)
        test_loss += loss_fn(logits, target).item()
        
        pred = logits.data.max(1)[1]
        correct += pred.eq(target.data).sum()
    
    test_loss /= len(test_loader.dataset)
    
    # 可视化
    viz.line([[test_loss, correct/len(test_loader.dataset)]], [global_step], win='test', 
           update='append')
    viz.images(data.view(-1, 1, 28, 28), win='x')
    viz.text(str(pred.detach().cpu().numpy()), win='pred',
           opts=dict(title='pred'))

    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))

Setting up a new session...
C:\Users\26250\AppData\Local\Temp/ipykernel_24916/2925979455.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(x)


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.310398
Train Epoch: 0 [25600/60000 (43%)]	Loss: 2.310926
Train Epoch: 0 [51200/60000 (85%)]	Loss: 2.308123

Test set: Average loss: 0.0092, Accuracy: 1006/10000 (10%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.304286
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.304272
Train Epoch: 1 [51200/60000 (85%)]	Loss: 2.299964

Test set: Average loss: 0.0092, Accuracy: 1446/10000 (14%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.298280
Train Epoch: 2 [25600/60000 (43%)]	Loss: 2.294664
Train Epoch: 2 [51200/60000 (85%)]	Loss: 2.293180

Test set: Average loss: 0.0092, Accuracy: 2008/10000 (20%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.290156
Train Epoch: 3 [25600/60000 (43%)]	Loss: 2.290926
Train Epoch: 3 [51200/60000 (85%)]	Loss: 2.282468

Test set: Average loss: 0.0091, Accuracy: 2379/10000 (24%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 2.280280
Train Epoch: 4 [25600/60000 (43%)]	Loss: 2.282762
Train Epoch: 4 [51200/60000 (85%)]	Loss: 2.271773

Test set: Average loss: 0.0091

In [6]:
# example for visdom
# lines : single trace

# viz = Visdom()

# viz.line([0.], [0.], win='train_loss', opts=dict(title='train_loss'))
# viz.line([loss.item()], [global_step], win='train_loss', update='append')

# lines : multi-traces

# viz.line([[0.0, 0.0]], [0.], win='test', opts=dict(title='test_loss & acc.', 
#                                                    legend=['loss', 'acc.']))
# viz.line([[test_loss, correct/len(test_loader.dataset)]], [global_step], win='test', 
#            update='append')


# viz.images(data.view(-1, 1, 28, 28), win='x')
# viz.text(str(pred.detach().cpu().numpy()), win='pred',
#          opts=dict(title='pred'))
